<a href="https://colab.research.google.com/github/prashantiyaramareddy/AI-ML-Learnings/blob/master/ComputerVision/NN_wandb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install wandb

In [3]:
import wandb
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint

import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras

In [7]:
sweep_config = {

    'method' : 'grid',
    'metric' : {
        'name': 'val_accuracy',
        'goal': 'maximize'
                },
    'parameters' : {
        'batch_size': {'values': [8,16]},
        'learning_rate': {'values': [0.001,0.0001]},
        'hidden_nodes': {'values': [128,64]},
        'img_size': {'values': [16, 224]},
        'epochs': {'values': [5,10]}
    }
}

sweep_id = wandb.sweep(sweep_config, project="5-flowers-new-sep-7")

Create sweep with ID: ut3hdo0r
Sweep URL: https://wandb.ai/prashantiyaramareddy7-nit-rourkela/5-flowers-new-sep-7/sweeps/ut3hdo0r


In [8]:
def train():
  with wandb.init() as run:
    config = wandb.config

    # Constants
    IMG_HEIGHT = config.img_size
    IMG_WIDTH = config.img_size
    IMG_CHANNELS = 3
    CLASS_NAMES = ["daisy", "dandelion", "roses", "sunflowers", "tulips"]

    def read_and_decode(filename, resize_dims):
        img_bytes = tf.io.read_file(filename)
        img = tf.image.decode_jpeg(img_bytes, channels=IMG_CHANNELS)
        img = tf.image.convert_image_dtype(img, tf.float32)
        img = tf.image.resize(img, resize_dims)
        return img

    def parse_csvline(csv_line):
        record_default = ["", ""]
        filename, label_string = tf.io.decode_csv(csv_line, record_default)

        img = read_and_decode(filename, [IMG_HEIGHT, IMG_WIDTH])

        # Convert label string to an integer index
        label = tf.where(tf.equal(CLASS_NAMES, label_string))[0, 0]

        return img, label

    # Define datasets
    train_dataset = (
        tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/train_set.csv")
        .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(config.batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )

    eval_dataset = (
        tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/eval_set.csv")
        .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(config.batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )

    #architecture
    model = keras.Sequential([
        keras.layers.Flatten(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)),
        keras.layers.Dense(config.hidden_nodes, activation="relu"),
        keras.layers.Dense(len(CLASS_NAMES), activation="softmax")
    ])

    model.compile(
          optimizer="adam",
          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
          metrics=["accuracy"]
      )

    model.fit(
      train_dataset,
      validation_data=eval_dataset,
      epochs=config.epochs,
      callbacks=[WandbMetricsLogger(log_freq=5)]
      )

In [ ]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: vvc5v69o with config:
wandb: 	batch_size: 8
wandb: 	epochs: 5
wandb: 	hidden_nodes: 128
wandb: 	img_size: 16
wandb: 	learning_rate: 0.001
wandb: Currently logged in as: prashantiyaramareddy7 (prashantiyaramareddy7-nit-rourkela) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch 1/5


/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


    413/Unknown 194s 466ms/step - accuracy: 0.3304 - loss: 1.5016

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


...

wandb: Agent Starting Run: jvh3lu0f with config:
wandb: 	batch_size: 8
wandb: 	epochs: 5
wandb: 	hidden_nodes: 128
wandb: 	img_size: 16
wandb: 	learning_rate: 0.0001


Epoch 1/5
413/413 ━━━━━━━━━━━━━━━━━━━━ 105s 252ms/step - accuracy: 0.3418 - loss: 1.5012 - val_accuracy: 0.4459 - val_loss: 1.2618
Epoch 2/5
413/413 ━━━━━━━━━━━━━━━━━━━━ 106s 255ms/step - accuracy: 0.4470 - loss: 1.2547 - val_accuracy: 0.4811 - val_loss: 1.2281
Epoch 3/5
413/413 ━━━━━━━━━━━━━━━━━━━━ 107s 260ms/step - accuracy: 0.4747 - loss: 1.1972 - val_accuracy: 0.4919 - val_loss: 1.2073
Epoch 4/5
413/413 ━━━━━━━━━━━━━━━━━━━━ 111s 269ms/step - accuracy: 0.5028 - loss: 1.1509 - val_accuracy: 0.4865 - val_loss: 1.2018
Epoch 5/5
413/413 ━━━━━━━━━━━━━━━━━━━━ 111s 269ms/step - accuracy: 0.5357 - loss: 1.1057 - val_accuracy: 0.4919 - val_loss: 1.1999


batch/accuracy,▁▁▂▂▃▃▄▄▄▄▄▄▅▅▅▆▆▆▆▆▆▆▇▇▇▆▆▆▆▇▇▇▇▇▇▇▇███
batch/batch_step,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▇▇▇▇███
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,█▇▆▅▅▅▁▃▃▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁
epoch/accuracy,▁▄▆▇█
epoch/epoch,▁▃▅▆█
epoch/learning_rate,▁▁▁▁▁
epoch/loss,█▅▃▂▁
epoch/val_accuracy,▁▆█▇█
epoch/val_loss,█▄▂▁▁
batch/accuracy,0.54045


wandb: Agent Starting Run: ld7cpwtp with config:
wandb: 	batch_size: 8
wandb: 	epochs: 5
wandb: 	hidden_nodes: 128
wandb: 	img_size: 224
wandb: 	learning_rate: 0.001


Epoch 1/5
     84/Unknown 39s 442ms/step - accuracy: 0.2520 - loss: 33.2298